In [2]:
library(fastDummies)
library(cplm)
library(tweedie)
library(statmod)
library(HDtweedie)
library(TDboost)

In [3]:
data(AutoClaim)
data <- AutoClaim
subset <- data[,"IN_YY"]==TRUE
data <- data[subset,]

n <- dim(data)[1]
y <- data[,"CLM_AMT5"]/1000
x1 <- data[,"KIDSDRIV"]

x2 <- data[,"TRAVTIME"]
x2 <- scale(x2)

x3<-data[,'CAR_USE']
z3<-dummy_columns(x3,remove_first_dummy=T)
z3<-z3[,2]

x4 <- data[,"BLUEBOOK"]
x4 <- scale(log(x4))
x5 <- data[,"NPOLICY"]

# create a group categorical variables
x6 <- data[,"CAR_TYPE"]
z6<-dummy_columns(x6,remove_first_dummy=T)
z6<-z6[,-1]
#ng <- length(levels(x6))
#name=c()
#for (i in 2:ng) {
#  name[i-1] <- paste("CAR_TYPE",i,sep="_")
#}
#colnames(z6) <- name


x7 <- data[,"RED_CAR"]
z7<-dummy_columns(x7,remove_first_dummy=T)
z7=z7[,-1]
x8 <- data[,"REVOLKED"]
z8<-dummy_columns(x8,remove_first_dummy=T)
z8=z8[,-1]

x9 <- data[,"MVR_PTS"]
x10 <- scale(data[,"AGE"])
x11 <- data[,"HOMEKIDS"]

x12 <- data[,"YOJ"]
me <- mean(x12,na.rm=T)
x12[is.na(x12)] <- me
x12 <- scale(x12)

x13 <- data[,"INCOME"]
x13 <- scale(log(x13+10))

x14 <- data[,"GENDER"]
z14=dummy_columns(x14,remove_first_dummy=T)
z14=z14[,-1]

x15 <- data[,"MARRIED"]
z15=dummy_columns(x15,remove_first_dummy=T)
z15=z15[,-1]

x16 <- data[,"PARENT1"]
z16=dummy_columns(x16,remove_first_dummy=T)
z16=z16[,-1]

# No observation in Unknown
x17 <- data[,"JOBCLASS"]
z17=dummy_columns(x17,remove_first_dummy=T)
z17=z17[,c(-1,-2)]

# ng <- length(levels(x17))
# name=c()
# for (i in 2:ng) {
#   name[i-1] <- paste("JOBCLASS",i,sep="_")
# }
# colnames(z17) <- name



#x17 <- as.numeric(x17)
#ng <- max(x17)
#name <- rep(NA,ng-2) # the category "Unknown" has no observations
#z17 <- matrix(NA,n,ng-2)
#for (i in 3:ng) {
#  z17[,i-2] <- as.numeric(x17==i)
#  name[i-2] <- paste("JOBCLASS",i,sep="_")
#}
#colnames(z17) <- name

x18 <- data[,"MAX_EDUC"]
z18=dummy_columns(x18,remove_first_dummy=T)
z18=z18[,-1]

# ng <- length(levels(x18))
# name=c()
# for (i in 2:ng) {
#   name[i-1] <- paste("MAX_EDUC",i,sep="_")
# }
# colnames(z18) <- name

#x18 <- as.numeric(x18)
#ng <- max(x18)
#name <- rep(NA,ng-1)
#z18 <- matrix(NA,n,ng-1)
#for (i in 2:ng) {
#  z18[,i-1] <- as.numeric(x18==i)
#  name[i-1] <- paste("MAX_EDUC",i,sep="_")
#}
#colnames(z18) <- name

x19 <- data[,"HOME_VAL"]
me <- median(x19,na.rm=T)
x19[is.na(x19)] <- me
x19 <- scale(x19)

x20 <- data[,"SAMEHOME"]
me <- mean(x20,na.rm=T)
x20[is.na(x20)] <- me
x20 <- scale(x20)

x21 <- data[,"AREA"]
z21=dummy_columns(x21,remove_first_dummy=T)
z21=z21[,-1]
#non-binary
z_nn <- cbind(x1,x2,x4,x5,x9,x10,x11,x12,x13,x19,x20)
name <- c("KIDSDRIV","TRAVTIME","BLUEBOOK","NPOLICY","MVR_PTS","AGE","HOMEKIDS","YOJ","INCOME","HOME_VAL","SAMEHOME")
colnames(z_nn) <- name

#binary
z_nb <- cbind(z3,z7,z8,z14,z15,z16,z21)
name <- c("CAR_USE","RED_CAR","REVOLKED","GENDER","MARRIED","PARENT1","AREA")
colnames(z_nb) <- name

# Categorical
z_c <- cbind(z6,z17,z18)
#x_c<- cbind(as.factor(x6),as.factor(x17),as.factor(x18))
#colnames(z_c)=c("CAR_TYPE","JOBCLASS","MAX_EDUC")

z_n <- cbind(z_nb,z_nn)
z0 <- cbind(z_c,z_n)

In [4]:
set.seed(9999)
sam <- sample(rep(1:2,len=n))
ytrain <- y[sam==1]
ytest <- y[sam==2]
z0=as.matrix(z0)
z0train <- z0[sam==1,]
z0test <- z0[sam==2,]

In [5]:
fit1 <- glm(ytrain~z0train,family=tweedie(link.power=0,var.power=1.7))
summary(fit1)
fit1_pred<-predict(fit1,as.data.frame(z0test),type='response')
mean((fit1_pred-ytest)^2)
mean(residuals(fit1)^2)


Call:
glm(formula = ytrain ~ z0train, family = tweedie(link.power = 0, 
    var.power = 1.7))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.5788  -2.8966  -2.5540   0.1732  10.1542  

Coefficients:
                            Estimate Std. Error t value Pr(>|t|)    
(Intercept)               -0.3953033  0.5341372  -0.740   0.4594    
z0train.data_Pickup       -0.1053282  0.3713660  -0.284   0.7767    
z0train.data_Sedan         0.2182657  0.3732561   0.585   0.5588    
z0train.data_Sports Car    0.2319588  0.4595823   0.505   0.6138    
z0train.data_SUV           0.1341331  0.4302914   0.312   0.7553    
z0train.data_Van           0.1903933  0.3840817   0.496   0.6202    
z0train.data_Clerical     -0.0107547  0.2458430  -0.044   0.9651    
z0train.data_Doctor       -0.4260808  0.5632756  -0.756   0.4495    
z0train.data_Home Maker   -0.5611354  0.3692932  -1.519   0.1289    
z0train.data_Lawyer       -0.2655183  0.3974679  -0.668   0.5042    
z0train.data_Mana

[1] 120.7862

[1] 6.53148

In [6]:
x_c<- cbind(as.factor(x6),as.factor(x17),as.factor(x18))
colnames(x_c)=c("CAR_TYPE","JOBCLASS","MAX_EDUC")
tdz0 <- cbind(x_c,z_n)
tdz0<-as.data.frame(tdz0)
tdz0_train=tdz0[sam==1,]
tdz0_test=tdz0[sam==2,]

autocross=tdz0_train
categorical_name<- c('CAR_TYPE','JOBCLASS','MAX_EDUC',"CAR_USE","RED_CAR","REVOLKED","GENDER","MARRIED","PARENT1","AREA")
autocross[,categorical_name]<-lapply(autocross[,categorical_name],factor)

autocross_test=tdz0_test
colnames(autocross_test)
autocross_test[,categorical_name]<-lapply(autocross_test[,categorical_name],factor)


summary(autocross)

[1] "CAR_TYPE" "JOBCLASS" "MAX_EDUC" "CAR_USE"  "RED_CAR"  "REVOLKED"
 [7] "GENDER"   "MARRIED"  "PARENT1"  "AREA"     "KIDSDRIV" "TRAVTIME"
[13] "BLUEBOOK" "NPOLICY"  "MVR_PTS"  "AGE"      "HOMEKIDS" "YOJ"     
[19] "INCOME"   "HOME_VAL" "SAMEHOME"

 CAR_TYPE    JOBCLASS   MAX_EDUC CAR_USE RED_CAR  REVOLKED GENDER  MARRIED
 1: 79    2      :319   1:204    0:921   0:1002   0:1227   0:772   0:565  
 2:241    3      :213   2:412    1:485   1: 404   1: 179   1:634   1:841  
 3:389    8      :202   3:447                                             
 4:172    7      :189   4:241                                             
 5:414    9      :161   5:102                                             
 6:111    6      :141                                                     
          (Other):181                                                     
 PARENT1  AREA        KIDSDRIV         TRAVTIME           BLUEBOOK        
 0:1218   0: 318   Min.   :0.0000   Min.   :-1.82782   Min.   :-3.411105  
 1: 188   1:1088   1st Qu.:0.0000   1st Qu.:-0.68493   1st Qu.:-0.554526  
                   Median :0.0000   Median :-0.04999   Median : 0.135923  
                   Mean   :0.2006   Mean   : 0.01730   Mean   :-0.000822  
                   3rd Qu

In [9]:
save(z0train,file="Tweedie_GLM_ALL_data_train.Rda")
save(z0test,file="Tweedie_GLM_ALL_data_test.Rda")
save(autocross,file="AutoCross_Data_train.Rda")
save(autocross_test,file="AutoCross_Data_test.Rda")
save(ytrain,file="Tweedie_GLM_ALL_data_y_train.Rda")
save(ytest,file="Tweedie_GLM_ALL_data_y_test.Rda")